<a href="https://colab.research.google.com/github/longvutr/Predict_QS/blob/main/Docking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#@title 1. Cài đặt Môi trường (Phiên bản Hoàn thiện)
%%shell
set -e

echo "--- Bước 1: Cài đặt các gói Python bằng pip (bao gồm cả thư viện vina) ---"
pip install vina rdkit chembl_webresource_client pandas py3Dmol > /dev/null

echo "\n--- Bước 2: Cài đặt Open Babel ---"
apt-get -qq update && apt-get -qq install openbabel > /dev/null

echo "\n✅✅✅ CÀI ĐẶT HOÀN TẤT! ✅✅✅"

--- Bước 1: Cài đặt các gói Python bằng pip (bao gồm cả thư viện vina) ---
\n--- Bước 2: Cài đặt Open Babel ---
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
\n✅✅✅ CÀI ĐẶT HOÀN TẤT! ✅✅✅


In [6]:
#@title 2. Chuẩn bị Receptor bằng Open Babel
import os
import numpy as np

!wget -q https://files.rcsb.org/download/6d6a.pdb -O 6d6a.pdb

ligand_coords = []
with open('6d6a.pdb', 'r') as f_in, open('receptor.pdb', 'w') as f_out:
    for line in f_in:
        if line.startswith('ATOM'):
            f_out.write(line)
        elif line.startswith('HETATM') and "FXD" in line:
            try:
                x = float(line[30:38]); y = float(line[38:46]); z = float(line[46:54])
                ligand_coords.append([x, y, z])
            except ValueError:
                continue

if not ligand_coords:
    raise ValueError("Không tìm thấy ligand FXD trong file 6d6a.pdb.")

center = np.mean(ligand_coords, axis=0).tolist()

print("\nBắt đầu chuẩn bị receptor bằng Open Babel...")
!obabel receptor.pdb -O receptor.pdbqt -xr

if os.path.exists('receptor.pdbqt'):
    print(f"\n✅ Receptor 'receptor.pdbqt' đã được chuẩn bị thành công.")
    print(f"✅ Trung tâm hộp docking (x, y, z) được tính toán: {center}")
    size_x, size_y, size_z = 25, 25, 25
else:
    print("\n❌ LỖI: Không thể tạo file receptor.pdbqt.")


Bắt đầu chuẩn bị receptor bằng Open Babel...
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is receptor.pdb)

1 molecule converted

✅ Receptor 'receptor.pdbqt' đã được chuẩn bị thành công.
✅ Trung tâm hộp docking (x, y, z) được tính toán: [45.77587500000003, 12.262112500000008, 38.08670833333331]


In [7]:
#@title 3. Lấy SMILES Tốc độ cao bằng Đa luồng
import pandas as pd
from chembl_webresource_client.new_client import new_client
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import re

# Dọn dẹp và chuẩn bị danh sách ID của bạn
chembl_ids_text = """
CHEMBL8CHEMBL424CHEMBL437CHEMBL8260CHEMBL9352CHEMBL8996CHEMBL66CHEMBL529CHEMBL532CHEMBL126CHEMBL127CHEMBL140CHEMBL27246CHEMBL148CHEMBL158CHEMBL44354CHEMBL262777
CHEMBL293492
CHEMBL70927CHEMBL311498
CHEMBL122890
CHEMBL356164
CHEMBL177CHEMBL374107
CHEMBL186CHEMBL244888
CHEMBL241149
CHEMBL397591
CHEMBL485818
CHEMBL506247
CHEMBL508112
CHEMBL1747CHEMBL1097933
CHEMBL1084380
CHEMBL1082934
CHEMBL1084054
CHEMBL1186500
CHEMBL1186528
CHEMBL1301126
CHEMBL1410311
CHEMBL1644450
CHEMBL1649757
CHEMBL1649771
CHEMBL1649772
CHEMBL1649773
CHEMBL1649774
CHEMBL1649775
CHEMBL1649776
CHEMBL1649777
CHEMBL1649778
CHEMBL1649788
CHEMBL1649789
CHEMBL1649790
CHEMBL1649791
CHEMBL1649792
CHEMBL1649793
CHEMBL1649794
CHEMBL1649795
CHEMBL1649796
CHEMBL1649797
CHEMBL1649798
CHEMBL1649799
CHEMBL1649800
CHEMBL1649801
CHEMBL1649802
CHEMBL1649803
CHEMBL1649804
CHEMBL1649805
CHEMBL1649806
CHEMBL1649807
CHEMBL1649808
CHEMBL1649809
CHEMBL1649810
CHEMBL1649811
CHEMBL1649812
CHEMBL1649813
CHEMBL1649814
CHEMBL1649815
CHEMBL1649816
CHEMBL1649817
CHEMBL1649818
CHEMBL1649819
CHEMBL1649820
CHEMBL1649821
CHEMBL1649822
CHEMBL1649823
CHEMBL1649824
CHEMBL1649825
CHEMBL1649826
CHEMBL1649827
CHEMBL1649828
CHEMBL1649829
CHEMBL1649830
CHEMBL1649831
CHEMBL1649832
CHEMBL1649833
CHEMBL1649834
CHEMBL1649835
CHEMBL1649836
CHEMBL1649838
CHEMBL1649839
CHEMBL1649840
CHEMBL1649842
CHEMBL1649843
CHEMBL1649844
CHEMBL1649845
CHEMBL1649846
CHEMBL1649847
CHEMBL1649848
CHEMBL1649849
CHEMBL1649850
CHEMBL1649851
CHEMBL1649852
CHEMBL1649853
CHEMBL1649854
CHEMBL1649855
CHEMBL1649856
CHEMBL1649857
CHEMBL1649858
CHEMBL1649859
CHEMBL1649587
CHEMBL1649860
CHEMBL1649861
CHEMBL1649862
CHEMBL1649863
CHEMBL1649864
CHEMBL1649865
CHEMBL1649866
CHEMBL1649867
CHEMBL1649868
CHEMBL1649869
CHEMBL1649870
CHEMBL1649871
CHEMBL1649872
CHEMBL1649873
CHEMBL1649874
CHEMBL1649875
CHEMBL1649876
CHEMBL1649877
CHEMBL1649878
CHEMBL1649879
CHEMBL1649880
CHEMBL1649881
CHEMBL1649882
CHEMBL1649883
CHEMBL1649884
CHEMBL1649885
CHEMBL1649886
CHEMBL1649887
CHEMBL1649888
CHEMBL1649889
CHEMBL1667926
CHEMBL1667927
CHEMBL1667928
CHEMBL1667929
CHEMBL1667930
CHEMBL1667932
CHEMBL1667933
CHEMBL1667934
CHEMBL1733252
CHEMBL1813999
CHEMBL1814000
CHEMBL1814001
CHEMBL1814002
CHEMBL1814003
CHEMBL1814004
CHEMBL1814005
CHEMBL1814006
CHEMBL1814007
CHEMBL1814008
CHEMBL1814009
CHEMBL1814010
CHEMBL1814011
CHEMBL1814012
CHEMBL1814013
CHEMBL1814014
CHEMBL1814015
CHEMBL1814016
CHEMBL1814017
CHEMBL1814018
CHEMBL1814019
CHEMBL1814020
CHEMBL1814021
CHEMBL1814022
CHEMBL1814023
CHEMBL1814024
CHEMBL1814025
CHEMBL1814026
CHEMBL1814027
CHEMBL1814028
CHEMBL1814029
CHEMBL1814030
CHEMBL1814031
CHEMBL1814032
CHEMBL1814033
CHEMBL1814034
CHEMBL1814035
CHEMBL1814041
CHEMBL1814042
CHEMBL1814208
CHEMBL1814209
CHEMBL1814210
CHEMBL1814211
CHEMBL1814212
CHEMBL1814213
CHEMBL1814214
CHEMBL1814215
CHEMBL1814216
CHEMBL1814217
CHEMBL1814218
CHEMBL1814219
CHEMBL1814220
CHEMBL1814221
CHEMBL1814222
CHEMBL1814223
CHEMBL1814224
CHEMBL1814225
CHEMBL1814226
CHEMBL1814227
CHEMBL1814238
CHEMBL1814239
CHEMBL1814240
CHEMBL1814241
CHEMBL1814242
CHEMBL1814244
CHEMBL1814245
CHEMBL1814246
CHEMBL1814247
CHEMBL1814248
CHEMBL1814249
CHEMBL1814250
CHEMBL1814251
CHEMBL1814252
CHEMBL1814253
CHEMBL1814254
CHEMBL1814255
CHEMBL1814256
CHEMBL1814257
CHEMBL1814258
CHEMBL1811944
CHEMBL1814259
CHEMBL1814260
CHEMBL1814263
CHEMBL1814365
CHEMBL1814366
CHEMBL1814367
CHEMBL1814368
CHEMBL1814369
CHEMBL1814370
CHEMBL1814371
CHEMBL1814372
CHEMBL1814373
CHEMBL1814374
CHEMBL1814375
CHEMBL1814376
CHEMBL1814377
CHEMBL1814378
CHEMBL1814379
CHEMBL1814380
CHEMBL1814381
CHEMBL1814382
CHEMBL1814383
CHEMBL1818523
CHEMBL1823930
CHEMBL1823942
CHEMBL1823945
CHEMBL2018916
CHEMBL2047546
CHEMBL2221249
CHEMBL2229607
CHEMBL2315299
CHEMBL2398107
"""
chembl_ids_full_list = [int(s.replace('CHEMBL', '')) for s in re.findall(r'CHEMBL\d+', chembl_ids_text)]

def fetch_smiles(chembl_id_int):
    molecule_api = new_client.molecule
    chembl_id = f"CHEMBL{chembl_id_int}"
    try:
        mol_data = molecule_api.filter(chembl_id=chembl_id).only(['molecule_structures'])[0]
        smiles = mol_data.get('molecule_structures', {}).get('canonical_smiles')
        if smiles:
            return {'status': 'success', 'id': chembl_id, 'smiles': smiles}
        else:
            return {'status': 'failed', 'id': chembl_id, 'error': 'Không có SMILES'}
    except Exception as e:
        return {'status': 'failed', 'id': chembl_id, 'error': str(e)}

print(f"🚀 BẮT ĐẦU LẤY SMILES CHO {len(chembl_ids_full_list)} LIGANDS BẰNG ĐA LUỒNG...")
start_time = time.time()
valid_ligands = []
failed_ligands_info = []
processed_count = 0

with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_id = {executor.submit(fetch_smiles, cid): cid for cid in chembl_ids_full_list}

    for future in as_completed(future_to_id):
        result = future.result()
        processed_count += 1
        progress = (processed_count / len(chembl_ids_full_list)) * 100

        if result['status'] == 'success':
            valid_ligands.append({'ChEMBL_ID': result['id'], 'SMILES': result['smiles']})
        else:
            failed_ligands_info.append(result)

        print(f"\r📊 Tiến trình: {processed_count}/{len(chembl_ids_full_list)} ({progress:.1f}%)", end="")

duration = time.time() - start_time
ligands_df = pd.DataFrame(valid_ligands)
ligands_df.to_csv('ligands.csv', index=False)

print("\n\n" + "="*60)
print("📊 BÁO CÁO TỔNG KẾT VIỆC LẤY SMILES")
print("="*60)
print(f"⏱️  Thời gian xử lý: {duration:.2f} giây")
print(f"✅ Thành công: {len(valid_ligands)}")
print(f"❌ Thất bại: {len(failed_ligands_info)}")
if failed_ligands_info:
    print("\n❌ CHI TIẾT CÁC LỖI GẶP PHẢI (hiển thị 10 lỗi đầu tiên):")
    for error in failed_ligands_info[:10]:
        print(f"  - {error['id']}: {error['error']}")
    if len(failed_ligands_info) > 10:
        print(f"  ... và {len(failed_ligands_info) - 10} lỗi khác.")
print(f"\n💾 Đã lưu các ligand hợp lệ vào file 'ligands.csv'.")

🚀 BẮT ĐẦU LẤY SMILES CHO 272 LIGANDS BẰNG ĐA LUỒNG...
📊 Tiến trình: 272/272 (100.0%)

📊 BÁO CÁO TỔNG KẾT VIỆC LẤY SMILES
⏱️  Thời gian xử lý: 1.51 giây
✅ Thành công: 272
❌ Thất bại: 0

💾 Đã lưu các ligand hợp lệ vào file 'ligands.csv'.


In [ ]:
#@title 4. Docking bằng thư viện `vina` (Nâng cấp)
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from vina import Vina
import time
import os

# KIỂM TRA QUAN TRỌNG: Đảm bảo Ô 2 đã được chạy
try:
    center
    size_x
except NameError:
    raise NameError("LỖI: Các biến 'center' và 'size' không tồn tại. Vui lòng chạy lại Ô 2 (Chuẩn bị Receptor) trước khi chạy ô này.")

if not os.path.exists('ligands.csv'):
    raise FileNotFoundError("Không tìm thấy file 'ligands.csv'. Vui lòng chạy lại ô số 3 trước.")

# Đọc danh sách ligand từ file CSV
ligands_to_dock_df = pd.read_csv('ligands.csv')
os.makedirs('ligands_pdbqt', exist_ok=True)
os.makedirs('docking_results', exist_ok=True)
results = []
start_time = time.time()

# --- SỬ DỤNG THƯ VIỆN VINA ---
# 1. Khởi tạo Vina một lần duy nhất
v = Vina(sf_name='vina')

# 2. Thiết lập receptor và tính toán bản đồ ái lực (grid maps) một lần
v.set_receptor('receptor.pdbqt')
v.compute_vina_maps(center=center, box_size=[size_x, size_y, size_z])
# --- KẾT THÚC THIẾT LẬP VINA ---

print(f"Bắt đầu quá trình docking cho {len(ligands_to_dock_df)} ligands...")

for index, row in ligands_to_dock_df.iterrows():
    chembl_id_str = row['ChEMBL_ID']
    smiles = row['SMILES']

    print(f"\n({index + 1}/{len(ligands_to_dock_df)}) Đang xử lý: {chembl_id_str}")

    try:
        # Chuẩn bị ligand PDBQT
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol, AllChem.ETKDGv3())
        AllChem.UFFOptimizeMolecule(mol)
        pdb_filename = f"temp_{chembl_id_str}.pdb"
        Chem.MolToPDBFile(mol, pdb_filename)
        ligand_pdbqt_path = f'ligands_pdbqt/{chembl_id_str}.pdbqt'
        !obabel {pdb_filename} -O {ligand_pdbqt_path} > /dev/null 2>&1
        os.remove(pdb_filename)

        # 3. Thiết lập ligand cho lần chạy này
        v.set_ligand_from_file(ligand_pdbqt_path)

        # 4. Thực hiện docking
        v.dock(exhaustiveness=8, n_poses=10) # Tạo 10 poses để tham khảo

        # 5. Lấy kết quả trực tiếp và lưu file PDBQT chứa các poses
        energies = v.energies(n_poses=10)
        best_energy = energies[0][0] # Năng lượng của pose tốt nhất (đầu tiên)

        output_file = f'docking_results/{chembl_id_str}_docked.pdbqt'
        v.write_poses(output_file, n_poses=10, overwrite=True)

        results.append({'ChEMBL ID': chembl_id_str, 'SMILES': smiles, 'Binding Affinity (kcal/mol)': best_energy})
        print(f"  -> Hoàn thành! Năng lượng tốt nhất: {best_energy:.3f} kcal/mol")

    except Exception as e:
        print(f"  -> Lỗi khi xử lý {chembl_id_str}: {e}")

end_time = time.time()
print(f"\nHoàn tất quá trình docking. Tổng thời gian: {(end_time - start_time)/60:.2f} phút.")

if not results:
    print("\n\n❌ KHÔNG CÓ KẾT QUẢ NÀO! Tất cả các lần docking đều thất bại.")
else:
    results_df = pd.DataFrame(results)
    results_df_sorted = results_df.sort_values(by='Binding Affinity (kcal/mol)', ascending=True)
    results_df_sorted.to_csv('docking_summary.csv', index=False)
    print("\n✅ Đã lưu thành công kết quả vào file 'docking_summary.csv'.")

Bắt đầu quá trình docking cho 272 ligands...

(1/272) Đang xử lý: CHEMBL8
  -> Hoàn thành! Năng lượng tốt nhất: -5.320 kcal/mol

(2/272) Đang xử lý: CHEMBL424
  -> Hoàn thành! Năng lượng tốt nhất: -4.752 kcal/mol

(3/272) Đang xử lý: CHEMBL437


In [ ]:
#@title 5. Phân tích và Hiển thị kết quả (Tương tác)
import pandas as pd
import py3Dmol
from IPython.display import display
import os

# --- Hiển thị bảng kết quả ---
if not os.path.exists('docking_summary.csv'):
    print("❌ Không tìm thấy file 'docking_summary.csv'. Có thể quá trình docking chưa được chạy hoặc đã gặp lỗi.")
else:
    results_df_sorted = pd.read_csv('docking_summary.csv')
    print("BẢNG KẾT QUẢ DOCKING (Sắp xếp theo năng lượng liên kết tốt nhất)")
    display(results_df_sorted)

    # --- Trực quan hóa 3D ---
    #@markdown ---
    #@markdown ###  Visualize a specific ligand
    #@markdown Nhập ChEMBL ID từ bảng trên (ví dụ: CHEMBL1234) rồi chạy lại ô này.
    chembl_id_to_show = "" #@param {type:"string"}

    if chembl_id_to_show:
        ligand_file_to_show = f'docking_results/{chembl_id_to_show}_docked.pdbqt'
        if os.path.exists(ligand_file_to_show):
            print(f"\nHiển thị tương tác của: {chembl_id_to_show}")
            view = py3Dmol.view(width=800, height=600)
            view.addModel(open('receptor.pdbqt', 'r').read(), 'pdbqt')
            view.setStyle({'model': -1}, {'cartoon': {'color': 'spectrum'}})
            view.addModel(open(ligand_file_to_show, 'r').read(), 'pdbqt')
            view.setStyle({'model': -1}, {'stick': {'colorscheme': 'cyanCarbon'}})
            view.zoomTo()
            view.show()
        else:
            print(f"❌ Không tìm thấy file kết quả cho '{chembl_id_to_show}'. Vui lòng kiểm tra lại ID.")
    else:
        print("\nNhập một ChEMBL ID vào ô ở trên và chạy lại cell này để xem hình ảnh 3D.")

In [ ]:
#@title 5. Phân tích và Hiển thị kết quả
import pandas as pd
import py3Dmol
from IPython.display import display
import os

if not os.path.exists('docking_summary.csv'):
    print("❌ Không tìm thấy file 'docking_summary.csv'. Có thể quá trình docking chưa được chạy hoặc đã gặp lỗi.")
else:
    results_df_sorted = pd.read_csv('docking_summary.csv')
    print("Top 20 ligands có ái lực liên kết tốt nhất:")
    display(results_df_sorted.head(20))

    if not results_df_sorted.empty:
        best_ligand_id = results_df_sorted.iloc[0]['ChEMBL ID']
        print(f"\nHiển thị tương tác của ligand tốt nhất: {best_ligand_id}")
        best_ligand_file = f'docking_results/{best_ligand_id}_out.pdbqt'

        if os.path.exists(best_ligand_file):
            view = py3Dmol.view(width=800, height=600)
            view.addModel(open('receptor.pdbqt', 'r').read(), 'pdbqt')
            view.setStyle({'model': -1}, {'cartoon': {'color': 'spectrum'}})
            view.addModel(open(best_ligand_file, 'r').read(), 'pdbqt')
            view.setStyle({'model': -1}, {'stick': {'colorscheme': 'cyanCarbon'}})
            view.zoomTo()
            view.show()
        else:
            print(f"❌ Không tìm thấy file kết quả PDBQT cho ligand tốt nhất: {best_ligand_file}")